In [ ]:
import os
import requests
import sqlite3
import pandas as pd
import io
import zipfile
from bs4 import BeautifulSoup
from unidecode import unidecode

# --- CONFIGURAÇÕES ---
URL_INMET = "https://portal.inmet.gov.br/dadoshistoricos"
DB_PATH = os.path.join('data', 'projeto_anac.db')
PREFIXO_ARQUIVO_ALVO = "PAMPULHA"

def padronizar_coluna(nome_original):
    nome_original = unidecode(nome_original).upper()
    
    # Mapeamento com as suas traduções em camelCase
    mapeamento = {
        # Datas e Horas
        'HORA (UTC)': 'hora',
        'HORA UTC': 'hora',
        'DATA (YYYY-MM-DD)': 'data',
        'DATA': 'data',
        
        # Pressão (Específicas primeiro)
        'PRESSAO ATMOSFERICA MAX': 'pressaoAtmosfericaMaxUltimaHora',
        'PRESSAO ATMOSFERICA MIN': 'pressaoAtmosfericaMinUltimaHora',
        'PRESSAO ATMOSFERICA AO NIVEL': 'pressaoAtmosferica',
        
        # Temperatura (Específicas primeiro)
        'TEMPERATURA ORVALHO MAX': 'temperaturaPontoOrvalhoMaxUltimaHora',
        'TEMPERATURA ORVALHO MIN': 'temperaturaPontoOrvalhoMinUltimaHora',
        'TEMPERATURA MAXIMA': 'temperaturaMaxUltimaHora',
        'TEMPERATURA MINIMA': 'temperaturaMinUltimaHora',
        'TEMPERATURA DO AR - BULBO SECO': 'temperaturaBulboSeco',
        'TEMPERATURA DO PONTO DE ORVALHO': 'temperaturaPontoOrvalho',
        
        # Umidade (Específicas primeiro)
        'UMIDADE REL. MAX': 'umidadeRelativaMaxUltimaHora',
        'UMIDADE REL. MIN': 'umidadeRelativaMinUltimaHora',
        'UMIDADE RELATIVA': 'umidadeRelativa',
        
        # Vento e Outros
        'VENTO, RAJADA': 'ventoRajadaMax',
        'VENTO, VELOCIDADE': 'ventoVelocidade',
        'VENTO, DIRECAO': 'ventoDirecaoGraus',
        'PRECIPITACAO': 'precipitacaoTotal',
        'RADIACAO': 'radiacaoGlobal'
    }

    for chave, nome_traduzido in mapeamento.items():
        if chave in nome_original:
            return nome_traduzido
            
    # Caso apareça algo novo, mantém um padrão camelCase básico
    return unidecode(nome_original).title().replace(' ', '').strip()


def processar_zip_inmet(url_zip, ano):
    try:
        print(f"  [INMET] Baixando ZIP de {ano}...")
        r = requests.get(url_zip, timeout=60) # Aumentado timeout para arquivos grandes
        r.raise_for_status()

        with zipfile.ZipFile(io.BytesIO(r.content)) as z:
            lista_arquivos = z.namelist()
            arquivo_alvo = next((f for f in lista_arquivos if PREFIXO_ARQUIVO_ALVO in f), None)
            
            if arquivo_alvo:
                print(f"    [ACHOU] Extraindo: {arquivo_alvo}")
                with z.open(arquivo_alvo) as f:
                    # O INMET usa latin-1 e ;
                    df = pd.read_csv(f, sep=';', encoding='latin-1', skiprows=8)
                    
                    # --- PADRONIZAÇÃO DE COLUNAS ---
                    # 1. Remove colunas vazias (comum no final dos CSVs do INMET)
                    df = df.dropna(axis=1, how='all')
                    
                    # APLICA A NOVA PADRONIZAÇÃO (Chama a função para cada coluna)
                    df.columns = [padronizar_coluna(c) for c in df.columns]
                    
                    df['anoRef'] = ano

                    df = df.replace(',', '.')
                    
                    with sqlite3.connect(DB_PATH) as conn:
                        # Agora o append funcionará pois as colunas são idênticas
                        df.to_sql('clima_inmet', conn, if_exists='append', index=False)
                    
                    print(f"    [SQLITE] {len(df)} linhas inseridas com sucesso.")
            else:
                print(f"    [AVISO] Estação não encontrada no ZIP de {ano}.")

    except Exception as e:
        print(f"    [ERRO] Falha no ano {ano}: {e}")


def extrair_dados_inmet():
    print(f"\n--- Iniciando Extração INMET ---")
    try:
        # Nota: O portal do INMET às vezes bloqueia requests sem User-Agent
        headers = {'User-Agent': 'Mozilla/5.0'}
        response = requests.get(URL_INMET, headers=headers, timeout=20)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Encontra todos os links que terminam em .zip
        links = soup.find_all('a', href=True)
        zips_encontrados = [l['href'] for l in links if l['href'].endswith('.zip')]
        
        print(f"[LOG] Encontrados {len(zips_encontrados)} arquivos anuais.")

        for url_zip in zips_encontrados:
            # Extrai o ano do nome do arquivo (ex: 2023.zip -> 2023)
            ano = url_zip.split('/')[-1].replace('.zip', '')    
            processar_zip_inmet(url_zip, ano)

    except Exception as e:
        print(f"[ERRO CRÍTICO INMET] {e}")

if __name__ == "__main__":
    # Garante pasta de dados
    os.makedirs('data', exist_ok=True)
    
    # Inicia a automação
    extrair_dados_inmet()



--- Iniciando Extração INMET ---
[LOG] Encontrados 27 arquivos anuais.
  [INMET] Baixando ZIP de 2000...
    [AVISO] Estação não encontrada no ZIP de 2000.
  [INMET] Baixando ZIP de 2001...
    [AVISO] Estação não encontrada no ZIP de 2001.
  [INMET] Baixando ZIP de 2002...
    [AVISO] Estação não encontrada no ZIP de 2002.
  [INMET] Baixando ZIP de 2003...
    [AVISO] Estação não encontrada no ZIP de 2003.
  [INMET] Baixando ZIP de 2004...
    [AVISO] Estação não encontrada no ZIP de 2004.
  [INMET] Baixando ZIP de 2005...
    [AVISO] Estação não encontrada no ZIP de 2005.
  [INMET] Baixando ZIP de 2006...
    [ACHOU] Extraindo: 2006/INMET_SE_MG_A521_PAMPULHA_10-10-2006_A_31-12-2006.CSV
    [SQLITE] 1992 linhas inseridas com sucesso.
  [INMET] Baixando ZIP de 2007...
    [ACHOU] Extraindo: 2007/INMET_SE_MG_A521_PAMPULHA_01-01-2007_A_31-12-2007.CSV
    [SQLITE] 8760 linhas inseridas com sucesso.
  [INMET] Baixando ZIP de 2008...
    [ACHOU] Extraindo: 2008/INMET_SE_MG_A521_PAMPULHA_01

In [4]:
import pandas as pd
df = pd.read_csv(r"C:\Users\Pichau\Downloads\VRA_20001.csv", skiprows=1, sep=';')

In [9]:
df.head()

,ICAO Empresa Aérea,Número Voo,Código Autorização (DI),Código Tipo Linha,ICAO Aeródromo Origem,ICAO Aeródromo Destino,Partida Prevista,Partida Real,Chegada Prevista,Chegada Real,Situação Voo,Código Justificativa
0,TAM,4660,0,R,SBCX,SBCT,01/01/2000 05:45,NaN,01/01/2000 06:37,NaN,CANCELADO,XS
1,VRG,8710,0,I,MMMX,SBEG,01/01/2000 19:00,NaN,02/01/2000 01:25,NaN,REALIZADO,NaN
2,BLC,8250,0,N,SBBR,SBGO,01/01/2000 20:04,NaN,01/01/2000 20:30,NaN,REALIZADO,NaN
3,BLC,241,9,R,SBSP,SBKP,NaN,01/01/2000 22:06,NaN,01/01/2000 22:45,REALIZADO,NaN
4,RSL,7161,9,R,SBRJ,SBCF,NaN,01/01/2000 10:46,NaN,01/01/2000 11:35,REALIZADO,NaN


In [8]:
for col in df.columns:
    print(col)

ICAO Empresa Aérea
Número Voo
Código Autorização (DI)
Código Tipo Linha
ICAO Aeródromo Origem
ICAO Aeródromo Destino
Partida Prevista
Partida Real
Chegada Prevista
Chegada Real
Situação Voo
Código Justificativa


In [ ]:
'ICAO Empresa Aérea',
'Número Voo',
'Código Autorização (DI)',
'Código Tipo Linha',
'ICAO Aeródromo Origem',
'ICAO Aeródromo Destino',
'Partida Prevista',
'Partida Real',
'Chegada Prevista',
'Chegada Real',
'Situação Voo',
'Código Justificativa'